In [1]:
import sys
sys.path.append("/home/pervinco/Upstage_Ai_Lab/Final/IR/src")

import os
import time
import json
import random
import warnings
import anthropic
import threading
import huggingface_hub

from tqdm import tqdm
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_text_splitters import RecursiveCharacterTextSplitter

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

from dotenv import load_dotenv
load_dotenv("../keys.env")

upstage_api_key = os.getenv("UPSTAGE_API_KEY")
os.environ['UPSTAGE_API_KEY'] = upstage_api_key

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = anthropic_api_key

hf_token = os.getenv("HF_TOKEN")
huggingface_hub.login(hf_token)

from config import Args
from data.data import load_document
from dense_retriever.model import load_dense_model
from sparse_retriever.model import load_sparse_model

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/pervinco/.cache/huggingface/token
Login successful


In [2]:
args = Args()

total_documents = load_document(path="../dataset/processed_documents.jsonl")
print(len(total_documents))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = args.chunk_size,
    chunk_overlap  = args.chunk_overlap,
    length_function = len,
)

4272


In [6]:
client = anthropic.Anthropic(api_key=anthropic_api_key)

DOCUMENT_CONTEXT_PROMPT = """
<document>
{doc_content}
</document>
"""

CHUNK_CONTEXT_PROMPT = """
전체 문서 내에 배치하려는 청크는 다음과 같습니다.
<chunk>
{chunk_content}
</chunk>

이 청크가 전체 문서에서 어떤 맥락에 속하는지 설명하는 간결한 문맥을 한국어로 작성하세요. 답변은 이 청크에 관한 짧고 구체적인 배경 설명을 포함해야 하며, 청크가 문서의 어느 부분에서 나온 것인지에 대한 정보를 제공해야 합니다.

    입력 예시:
        회사의 매출이 전 분기 대비 3% 증가했습니다.

    주어진 청크 예시에서는 '회사'가 어떤 회사를 말하는 것인지, '전 분기'가 정확하게 몇년도 몇분기에 대한 것인지 정보가 포함되어 있지 않습니다. 따라서 당신은 아래 출력 예시처럼 입력되는 청크를 읽고 정보검색에 유용하도록 더 명확한 청크로 재구성해야합니다.

    출력 예시: 
        이 청크는 2023년 2분기에 ACME 회사의 실적을 다룬 SEC 보고서에서 발췌되었습니다. 이전 분기의 수익은 3억 1천 4백만 달러였으며, 회사의 수익은 이전 분기 대비 3% 증가했습니다.
"""

token_counts = {
    'input': 0,
    'output': 0,
    'cache_read': 0,
    'cache_creation': 0
}
token_lock = threading.Lock()

In [4]:
def situate_context(doc: str, chunk: str, max_retries=5) -> str:
    for attempt in range(max_retries):
        try:
            response = client.beta.prompt_caching.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=1024,
                temperature=0.0,
                messages=[
                    {
                        "role": "user", 
                        "content": [
                            {
                                "type": "text",
                                "text": DOCUMENT_CONTEXT_PROMPT.format(doc_content=doc),
                                "cache_control": {"type": "ephemeral"}
                            },
                            {
                                "type": "text",
                                "text": CHUNK_CONTEXT_PROMPT.format(chunk_content=chunk),
                            }
                        ]
                    }
                ],
                extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
            )
            with token_lock:
                token_counts['input'] += response.usage.input_tokens
                token_counts['output'] += response.usage.output_tokens
                token_counts['cache_read'] += response.usage.cache_read_input_tokens
                token_counts['cache_creation'] += response.usage.cache_creation_input_tokens
            return response
        except anthropic.RateLimitError as e:
            if attempt == max_retries - 1:
                raise
            wait_time = (2 ** attempt) + (random.random() * 0.1)
            print(f"Rate limit hit. Waiting for {wait_time:.2f} seconds before retry.")
            time.sleep(wait_time)

In [5]:
def process_chunk(document, chunk):
    result = situate_context(document.page_content, chunk)
    return {
        "docid": document.metadata['docid'],
        "content": f"{chunk}\n\n{result.content[0].text}"
    }

In [6]:
def process_documents(documents, text_splitter, output_file, parallel_threads=5):
    with open(output_file, 'w', encoding='utf-8') as f:
        with ThreadPoolExecutor(max_workers=parallel_threads) as executor:
            for document in tqdm(documents, desc="Processing documents"):
                chunks = text_splitter.split_text(document.page_content)
                futures = [executor.submit(process_chunk, document, chunk) for chunk in chunks]
                
                for future in as_completed(futures):
                    try:
                        result = future.result()
                        f.write(json.dumps(result, ensure_ascii=False) + '\n')
                    except Exception as e:
                        print(f"Error processing chunk: {e}")
                
                time.sleep(random.uniform(1, 2))

In [ ]:
output_file = '../dataset/antropic_contextual_retrieval_documents.jsonl'

# 중단 지점 저장 및 불러오기
try:
    with open('progress.json', 'r') as f:
        progress = json.load(f)
        start_index = progress['last_processed_index'] + 1
except FileNotFoundError:
    start_index = 0

try:
    process_documents(total_documents[start_index:], text_splitter, output_file)
except KeyboardInterrupt:
    print("작업이 중단되었습니다. 진행 상황을 저장합니다.")
finally:
    with open('progress.json', 'w') as f:
        json.dump({'last_processed_index': start_index + len(total_documents) - 1}, f)

# 토큰 사용량 출력
print(f"Total input tokens: {token_counts['input']}")
print(f"Total output tokens: {token_counts['output']}")
print(f"Total tokens read from cache: {token_counts['cache_read']}")
print(f"Total tokens written to cache: {token_counts['cache_creation']}")

total_tokens = token_counts['input'] + token_counts['cache_read'] + token_counts['cache_creation']
savings_percentage = (token_counts['cache_read'] / total_tokens) * 100 if total_tokens > 0 else 0
print(f"Total input token savings from prompt caching: {savings_percentage:.2f}% of all input tokens used were read from cache.")

In [3]:
client = OpenAI()
model = "gpt-4o"

# client = OpenAI(
#     api_key=upstage_api_key,
#     base_url="https://api.upstage.ai/v1/solar"
# )
# model = "solar-pro"

In [4]:
prompt = """
<document>
{DOCUMENT}
</document> 
전체 문서에서 발췌한 청크는 다음과 같습니다.
<chunk> 
{CHUNK}
</chunk>

이 청크가 전체 문서의 어떤 맥락에 속하는지 한국어로 간결하게 설명하세요. 청크가 문서의 어떤 부분에서 발췌되었는지에 대한 정보를 제공하고, 청크의 배경 설명을 명확하게 해주세요.

입력 예시:
    건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다.
예시에 대한 설명:
    이 청크는 건강한 생활습관과 관련된 영양학 문서에서 발췌되었으며, 에너지 섭취와 소비의 균형을 유지하는 방법에 대한 설명입니다. 이 설명은 특히 식단과 운동을 통한 에너지 조절의 중요성에 초점을 맞추고 있습니다.
출력 예시:
    이 청크는 영양학과 관련된 2024년 연구 보고서에서 발췌되었습니다. 이 문서에서는 에너지 균형을 유지하는 것이 건강한 생활에 얼마나 중요한지 설명하고 있으며, 특히 1-2주 동안의 에너지 섭취와 소비 조절을 강조하고 있습니다.
"""

In [5]:
def gpt_contextual_retrieval(document, chunk, model: str, client: OpenAI):
    prompt = """
    <document>
    {DOCUMENT}
    </document> 
    전체 문서에서 발췌한 청크는 다음과 같습니다.
    <chunk> 
    {CHUNK}
    </chunk>

    1.주어진 청크에 대한 제목, 요약, 여러 개의 가설적 질문 등 다양한 정보들을 생성해주세요.
    2.이 청크가 전체 문서의 어떤 맥락에 속하는지 한국어로 간결하게 설명하세요. 청크가 문서의 어떤 부분에서 발췌되었는지에 대한 정보를 제공하고, 청크의 배경 설명을 명확하게 해주세요.
    """
    prompt = prompt.format(DOCUMENT=document, CHUNK=chunk)
    
    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1
            )
            return completion.choices[0].message.content
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts: {e}")
                return None
            time.sleep(2 ** attempt + random.random())


In [6]:
def process_chunk(args):
    document, chunk, model, client = args
    result = gpt_contextual_retrieval(document.page_content, chunk, model, client)
    if result is not None:
        return {
            "docid": document.metadata['docid'],
            "content": f"{chunk}\n\n{result}"
        }
    return None


def process_documents(documents, text_splitter, output_file, max_workers=5):
    with open(output_file, 'w', encoding='utf-8') as f:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for document in tqdm(documents, desc="Processing documents"):
                chunks = text_splitter.split_text(document.page_content)
                futures = [executor.submit(process_chunk, (document, chunk, model, client)) for chunk in chunks]
                
                for future in as_completed(futures):
                    result = future.result()
                    if result is not None:
                        f.write(json.dumps(result, ensure_ascii=False) + '\n')
                
                time.sleep(random.uniform(1, 2))  # 문서 간 1~2초 랜덤 대기

In [7]:
output_file = '../dataset/gpt_contextual_retrieval_documents_v4.jsonl'
process_documents(total_documents, text_splitter, output_file)

# try:
#     with open('progress.json', 'r') as f:
#         progress = json.load(f)
#         start_index = progress['last_processed_index'] + 1
# except FileNotFoundError:
#     start_index = 0

# try:
#     process_documents(total_documents[start_index:], text_splitter, output_file)
# except KeyboardInterrupt:
#     print("작업이 중단되었습니다. 진행 상황을 저장합니다.")
# finally:
#     with open('progress.json', 'w') as f:
#         json.dump({'last_processed_index': start_index + len(total_documents) - 1}, f)

Processing documents: 100%|██████████| 4272/4272 [12:12:42<00:00, 10.29s/it]   
